# LOICA Consortia demonstration

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

This notebook shows the novel functionality that has been encoded in git branch Consortia

This LOICA update allows to create strains that belong to class Strain, taking genetic network and metabolism as parameters.

## Test producer strain
First, set up metabolism and genetic network as usual

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab1 = lc.SimulatedMetabolism("Simulated metabolism 1", biomass, growth_rate)
genetic_n1 = lc.GeneticNetwork()

Then define strain

In [ ]:
test_strain = lc.Strain('Test strain', genetic_n1, metab1)
# test
print(test_strain.name)

Now, create genetic network as usual. 

### No diffusion

Each gene product now has parameter diffusion_rate, which by default is set to 0. 

LOICA Consortia assumes that:
- all molecules are distrubed equally both incracellularly and extracellularly, so the only diffusion gradient exists between extracellular space and inside of the cell. 
- all cells belonging to the same strain are of uniform size. 

Based on that, diffusion rate here is the proportion of concentration/molecule difference that gets transferred either in- or outside of the cell per change of time. This parameter depend on the average surface area of the cell in the strain.

In [ ]:
# Create and add reporter to genetic network
reporter = lc.Reporter(name='GFP', color='green', degradation_rate=0, init_concentration=0)
genetic_n1.add_reporter(reporter)

# Constitutive exression of reporter from strain
source = lc.Source(output=reporter, rate=1000)
genetic_n1.add_operator(source)

Now we have a strain that constitutively expresses GFP. 

Since we have not set diffusion rate, if we create sample, assay and run the latter then check the reporter extracellular concentration, it will stay 0:

In [ ]:
sample = lc.Sample(strain=test_strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='GFP constittive expression',
                description='Simulated GFP expression in test strain'
                )

Setting parameter track_all to true will record both intracellular and extracellular concentrations of all GeneProducts in the systems, as well as biomass without accounting for noise and background.

In [ ]:
assay.run(stochastic=False, track_all=True)

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Test strain'].plot(x='Time', y='Measurement', style='-', ax=ax)
ax2 = ax.twinx()
m[m.Signal=='Test strain biomass'].plot(x='Time', y='Measurement', style='-', ax=ax2)
plt.legend(['Extracellular GFP', 'Internal GFP', 'Biomass'])

### With diffusion

Now let's add diffusible signal C14 with diffusion rate 1 (all difference gets trasferred)that would also be continiously expressed.

In [ ]:
regulator = lc.Regulator(name='C14', degradation_rate=0, init_concentration=0, diffusion_rate=1)
genetic_n1.add_regulator(regulator)

source_2 = lc.Source(output=regulator, rate=1000)
genetic_n1.add_operator(source_2)

In [ ]:
sample = lc.Sample(strain=test_strain)
assay2 = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='GFP and C14 constittive expression',
                description='Simulated GFP and C14 expression in test strain'
                )

In [ ]:
assay2.run(stochastic=False, track_all=True)

In [ ]:
m = assay2.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Test strain'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='C14 in Test strain'].plot(x='Time', y='Measurement', style='-', ax=ax)
ax2 = ax.twinx()
m[m.Signal=='Test strain biomass'].plot(x='Time', y='Measurement', style='-', ax=ax2)
plt.legend(['Extracellular GFP', 'Internal GFP', 'Extracellular C14', 'C14 in Test strain', 'Biomass'])

### Extracellular degradation

It is also possible to set extracellular degradation rate for any GeneProduct.

In [ ]:
sample.set_extracel_degr("C14", 0.5)
assay3 = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='GFP and C14 constittive expression with C14 extracellular degradation',
                description='Simulated GFP and C14 expression in test strain'
                )

In [ ]:
assay3.run(stochastic=False, track_all=True)

In [ ]:
m = assay3.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Test strain'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='C14 in Test strain'].plot(x='Time', y='Measurement', style='-', ax=ax)
ax2 = ax.twinx()
m[m.Signal=='Test strain biomass'].plot(x='Time', y='Measurement', style='-', ax=ax2)
plt.legend(['Extracellular GFP', 'Internal GFP', 'Extracellular C14', 'C14 in Test strain', 'Biomass'])

## Sender-Receiver consortium

### Sender strain

In [ ]:
def growth_rate_sender(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass_sender(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab_sender = lc.SimulatedMetabolism("Simulated metabolism receiver", biomass_sender, growth_rate_sender)
genetic_network_sender = lc.GeneticNetwork()
sender = lc.Strain('Sender', genetic_network_sender, metab_sender)
# test
print(sender.name)

In [ ]:
# Create and add reporter to genetic network
signal = lc.Regulator(name='C14', degradation_rate=0, init_concentration=0, diffusion_rate=1)
genetic_network_sender.add_regulator(signal)

# Constitutive exression of signal in sender
signal_source = lc.Source(output=signal, rate=1000)
genetic_network_sender.add_operator(signal_source)

### Receiver strain

In [ ]:
def growth_rate_receiver(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass_receiver(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab_receiver = lc.SimulatedMetabolism("Simulated metabolism receiver", biomass_receiver, growth_rate_receiver)
genetic_network_receiver = lc.GeneticNetwork()
receiver = lc.Strain('Receiver', genetic_network_receiver, metab_receiver)
# test
print(receiver.name)

When defining multiple strains, if molecule can diffuse to cell ("receiver") that doesn't belong to the producer strain, it has to be defined within the "receiver" strain as well. Since instance of GeneProduct has own internal concentration, assigning the same value for different genetic networks will not work, as different strains with different degetic networks will have different dynamics. 

In [ ]:
# Create and add regulator to genetic network
signal_in_r = lc.Regulator(name='C14', degradation_rate=0, init_concentration=0, diffusion_rate=1)
genetic_network_receiver.add_regulator(signal_in_r)

# Create and add reporter
gfp = lc.Reporter(name='GFP', degradation_rate=0, init_concentration=0)
genetic_network_receiver.add_reporter(gfp)

# Exression of GFP activated by C14
c14_to_gfp = lc.Hill1(name='Pcin', input=signal_in_r, output=gfp, alpha=[0, 100], K=10, n=2)
genetic_network_receiver.add_operator(c14_to_gfp)

### Set up sample and simulate

In [ ]:
s_r_consortium = lc.Sample(strain=[sender, receiver])
s_r_assay = lc.Assay([s_r_consortium], 
                n_measurements=100, 
                interval=0.24,
                name='GFP expression in receiver strain',
                description='Simulated sender-receiver consortium'
                )

In [ ]:
s_r_assay.run(stochastic=False, track_all=True)

In [ ]:
m = s_r_assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
# TODO: add so this would be recorder as well
m[m.Signal=='GFP in Sender'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Receiver'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', color='red', style='.', ax=ax)
m[m.Signal=='C14 in Sender'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='C14 in Receiver'].plot(x='Time', y='Measurement', color='red', style='--', ax=ax)
ax2 = ax.twinx()
m[m.Signal=='Sender biomass'].plot(x='Time', y='Measurement', color='black', style='-', ax=ax2)
m[m.Signal=='Receiver biomass'].plot(x='Time', y='Measurement', color='purple', style='-', ax=ax2)
plt.legend(['Extracellular GFP', 'GFP in Sender', 'GFP in Receiver', 'Extracellular C14', 'C14 in Sender', 'C14 in Receiver', 'Sender biomass', 'Receiver biomass'])

# Diffusible supplement

You can set a signal to be continuously added at the same concentration to the the sample. In the code below, we create receiver strain that produces GFP when C14 is present. Then we create supplement C14.

### Receiver strain

In [ ]:
def growth_rate_receiver(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass_receiver(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab_receiver = lc.SimulatedMetabolism("Simulated metabolism receiver", biomass_receiver, growth_rate_receiver)
genetic_network_receiver = lc.GeneticNetwork()
receiver = lc.Strain('Receiver', genetic_network_receiver, metab_receiver)
# test
print(receiver.name)

In [ ]:
# Create and add regulator to genetic network
signal = lc.Regulator(name='C14', degradation_rate=0, init_concentration=0, diffusion_rate=1)
genetic_network_receiver.add_regulator(signal_in_r)

# Create and add reporter
gfp = lc.Reporter(name='GFP', degradation_rate=0, init_concentration=0)
genetic_network_receiver.add_reporter(gfp)

# Exression of GFP activated by C14
c14_to_gfp = lc.Hill1(name='Pcin', input=signal, output=gfp, alpha=[0, 100], K=10, n=2)
genetic_network_receiver.add_operator(c14_to_gfp)

Add supplement. Make sure that the name is the same as the as the regulator name that belongs to the receiver strain.

In [ ]:
supplement = lc.Supplement(name='C14')

### Set up sample, set supplement and simulate

In [ ]:
receiver_sample = lc.Sample(strain=receiver)
receiver_sample.set_supplement(supplement, 10)
receiver_assay = lc.Assay([receiver_sample], 
                n_measurements=100, 
                interval=0.24,
                name='GFP expression in receiver strain based on supplement diffusion',
                description='Simulated receiver strain'
                )

In [ ]:
receiver_assay.run(stochastic=False, track_all=True)

In [ ]:
m = receiver_assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Receiver'].plot(x='Time', y='Measurement', color='green', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', color='red', style='.', ax=ax)
m[m.Signal=='C14 in Receiver'].plot(x='Time', y='Measurement', color='red', style='--', ax=ax)
ax2 = ax.twinx()
m[m.Signal=='Receiver biomass'].plot(x='Time', y='Measurement', color='purple', style='-', ax=ax2)
plt.legend(['Extracellular GFP', 'GFP in Receiver', 'Extracellular C14', 'C14 in Receiver', 'Receiver biomass'])

# Set initial external concentration

You can set a starting external concentration of signal. In the code below, we create receiver strain that produces GFP when C14 is present. Then we add C14 at the start of the experiment.

### Receiver strain

In [ ]:
def growth_rate_receiver(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass_receiver(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab_receiver = lc.SimulatedMetabolism("Simulated metabolism receiver", biomass_receiver, growth_rate_receiver)
genetic_network_receiver = lc.GeneticNetwork()
receiver = lc.Strain('Receiver', genetic_network_receiver, metab_receiver)
# test
print(receiver.name)

In [ ]:
# Create and add regulator to genetic network
signal = lc.Regulator(name='C14', degradation_rate=0, init_concentration=0, diffusion_rate=1)
genetic_network_receiver.add_regulator(signal_in_r)

# Create and add reporter
gfp = lc.Reporter(name='GFP', degradation_rate=0, init_concentration=0)
genetic_network_receiver.add_reporter(gfp)

# Exression of GFP activated by C14
c14_to_gfp = lc.Hill1(name='Pcin', input=signal, output=gfp, alpha=[0, 100], K=10, n=2)
genetic_network_receiver.add_operator(c14_to_gfp)

### Set up sample, set starting concentration and simulate

In [ ]:
receiver2_sample = lc.Sample(strain=receiver)
receiver2_sample.set_ext_conc('C14', 50)
print(signal.init_ext_conc)

In [ ]:
receiver2_assay = lc.Assay([receiver2_sample], 
                n_measurements=100, 
                interval=0.24,
                name='GFP expression in receiver strain based on C14 diffusion',
                description='Simulated receiver strain'
                )

In [ ]:
receiver2_assay.run(stochastic=False, track_all=True)

In [ ]:
m = receiver2_assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Receiver'].plot(x='Time', y='Measurement', color='green', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', color='red', style='.', ax=ax)
m[m.Signal=='C14 in Receiver'].plot(x='Time', y='Measurement', color='red', style='--', ax=ax)
ax2 = ax.twinx()
m[m.Signal=='Receiver biomass'].plot(x='Time', y='Measurement', color='purple', style='-', ax=ax2)
plt.legend(['Extracellular GFP', 'GFP in Receiver', 'Extracellular C14', 'C14 in Receiver', 'Receiver biomass'])

# Stochastic simulation

There are several options for simulating the model stochastically. The main (working) model is fully stochastic, with all reactions (including diffusion in and diffusion out) to be simulated according to Gillespie algorythm.

To simulate it, set up a consortium again.

## Fully stochastic

### Sender strain

In [ ]:
def growth_rate_sender(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass_sender(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab_sender = lc.SimulatedMetabolism("Simulated metabolism receiver", biomass_sender, growth_rate_sender)
genetic_network_sender = lc.GeneticNetwork()
sender = lc.Strain('Sender', genetic_network_sender, metab_sender)
# test
print(sender.name)

In [ ]:
# Create and add reporter to genetic network
signal = lc.Regulator(name='C14', degradation_rate=0, init_concentration=0, diffusion_rate=1)
genetic_network_sender.add_regulator(signal)

# Constitutive exression of signal in sender
signal_source = lc.Source(output=signal, rate=10)
genetic_network_sender.add_operator(signal_source)

### Receiver strain

In [ ]:
def growth_rate_receiver(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass_receiver(t):
    return lc.gompertz(t, 0.01, 1, 1, 0.5)

metab_receiver = lc.SimulatedMetabolism("Simulated metabolism receiver", biomass_receiver, growth_rate_receiver)
genetic_network_receiver = lc.GeneticNetwork()
receiver = lc.Strain('Receiver', genetic_network_receiver, metab_receiver)
# test
print(receiver.name)

In [ ]:
# Create and add regulator to genetic network
signal_in_r = lc.Regulator(name='C14', degradation_rate=0, init_concentration=0, diffusion_rate=1)
genetic_network_receiver.add_regulator(signal_in_r)

# Create and add reporter
gfp = lc.Reporter(name='GFP', degradation_rate=0, init_concentration=0)
genetic_network_receiver.add_reporter(gfp)

# Exression of GFP activated by C14
c14_to_gfp = lc.Hill1(name='Pcin', input=signal_in_r, output=gfp, alpha=[0, 100], K=10, n=2)
genetic_network_receiver.add_operator(c14_to_gfp)

### Set up sample and simulate

In [ ]:
stoch_consortium = lc.Sample(strain=[sender, receiver])
stoch_assay = lc.Assay([stoch_consortium], 
                n_measurements=100, 
                interval=0.24,
                name='GFP expression in receiver strain',
                description='Stochastically simulated sender-receiver consortium'
                )

In [ ]:
stoch_assay.run(stochastic=True, track_all=True)

In [ ]:
m = stoch_assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Sender'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Receiver'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', color='red', style='.', ax=ax)
m[m.Signal=='C14 in Sender'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='C14 in Receiver'].plot(x='Time', y='Measurement', color='red', style='--', ax=ax)
m[m.Signal=='Sender biomass'].plot(x='Time', y='Measurement', color='black', style='-', ax=ax)
m[m.Signal=='Receiver biomass'].plot(x='Time', y='Measurement', color='purple', style='-', ax=ax)
plt.legend(['Extracellular GFP', 'GFP in Sender', 'GFP in Receiver', 'Extracellular C14', 'C14 in Sender', 'C14 in Receiver', 'Sender biomass', 'Receiver biomass'])

## Stochastic with partitions

It is also possible to have stochastic simulation where reactions happen simultaneously in each strain and extracellular space (so there is a simultaneous Gillespie algorythm running in each compartment).

In [ ]:
stoch_comp_consortium = lc.Sample(strain=[sender, receiver])
stoch_comp_assay = lc.Assay([stoch_comp_consortium], 
                n_measurements=100, 
                interval=0.24,
                name='GFP expression in receiver strain',
                description='Stochastically simulated (compartments) sender-receiver consortium'
                )

In [ ]:
stoch_comp_assay.run(stochastic='full+comp', track_all=True)

In [ ]:
m = stoch_comp_assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Sender'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Receiver'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', color='red', style='.', ax=ax)
m[m.Signal=='C14 in Sender'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='C14 in Receiver'].plot(x='Time', y='Measurement', color='red', style='--', ax=ax)
m[m.Signal=='Sender biomass'].plot(x='Time', y='Measurement', color='black', style='-', ax=ax)
m[m.Signal=='Receiver biomass'].plot(x='Time', y='Measurement', color='purple', style='-', ax=ax)
plt.legend(['Extracellular GFP', 'GFP in Sender', 'GFP in Receiver', 'Extracellular C14', 'C14 in Sender', 'C14 in Receiver', 'Sender biomass', 'Receiver biomass'])

## Semi-stochastic (diffusion is deterministic)

Additionally, you can simulate semi-stochastic model with compartments: as above, reactions happen simultaneously in each strain and extracellular space, but diffusion is deterministic.

In [ ]:
semi_stoch_consortium = lc.Sample(strain=[sender, receiver])
semi_stoch_assay = lc.Assay([semi_stoch_consortium], 
                n_measurements=100, 
                interval=0.24,
                name='GFP expression in receiver strain',
                description='Semi-stochastically simulated sender-receiver consortium'
                )

In [ ]:
semi_stoch_assay.run(stochastic='semi+comp', track_all=True)

In [ ]:
m = semi_stoch_assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Sender'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Receiver'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', color='red', style='.', ax=ax)
m[m.Signal=='C14 in Sender'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='C14 in Receiver'].plot(x='Time', y='Measurement', color='red', style='--', ax=ax)
m[m.Signal=='Sender biomass'].plot(x='Time', y='Measurement', color='black', style='-', ax=ax)
m[m.Signal=='Receiver biomass'].plot(x='Time', y='Measurement', color='purple', style='-', ax=ax)
plt.legend(['Extracellular GFP', 'GFP in Sender', 'GFP in Receiver', 'Extracellular C14', 'C14 in Sender', 'C14 in Receiver', 'Sender biomass', 'Receiver biomass'])

## Deterministic for comparison

Simulate deterministic model under same conditions for comparison

In [ ]:
det_consortium = lc.Sample(strain=[sender, receiver])
det_assay = lc.Assay([stoch_consortium], 
                n_measurements=100, 
                interval=0.24,
                name='GFP expression in receiver strain',
                description='Stochastically simulated sender-receiver consortium'
                )

In [ ]:
det_assay.run(stochastic=False, track_all=True)

In [ ]:
m = det_assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='Extracellular GFP'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Sender'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='GFP in Receiver'].plot(x='Time', y='Measurement', style='-', ax=ax)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', color='red', style='.', ax=ax)
m[m.Signal=='C14 in Sender'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='C14 in Receiver'].plot(x='Time', y='Measurement', color='red', style='--', ax=ax)
m[m.Signal=='Sender biomass'].plot(x='Time', y='Measurement', color='black', style='-', ax=ax)
m[m.Signal=='Receiver biomass'].plot(x='Time', y='Measurement', color='purple', style='-', ax=ax)
plt.legend(['Extracellular GFP', 'GFP in Sender', 'GFP in Receiver', 'Extracellular C14', 'C14 in Sender', 'C14 in Receiver', 'Sender biomass', 'Receiver biomass'])